# 3 Pointers Made against advanced_totals.csv

### Import packages

In [8]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
from scipy.stats import pearsonr, zscore
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [9]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [10]:
advanced_df = pd.read_csv('backend/data/totals/advanced_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

In [11]:
# Merge dataframes to have target variable
df = pd.merge(shooting_df, advanced_df, 
              left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
              how='left')

### Basic exploration

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40218 entries, 0 to 40217
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      40218 non-null  object 
 1   visitor   40218 non-null  object 
 2   home      40218 non-null  object 
 3   team      40218 non-null  int64  
 4   3p        40202 non-null  float64
 5   ts_perc   40202 non-null  float64
 6   efg_perc  40202 non-null  float64
 7   3par      40202 non-null  float64
 8   ftr       40202 non-null  float64
 9   orb_perc  40202 non-null  float64
 10  drb_perc  40202 non-null  float64
 11  trb_perc  40202 non-null  float64
 12  ast_perc  40202 non-null  float64
 13  stl_perc  40202 non-null  float64
 14  blk_perc  40202 non-null  float64
 15  tov_perc  40202 non-null  float64
 16  usg_perc  40202 non-null  float64
 17  ortg      40202 non-null  float64
 18  drtg      40202 non-null  float64
dtypes: float64(15), int64(1), object(3)
memory usage: 6.1+ MB


In [13]:
df.head()

,date,visitor,home,team,3p,ts_perc,efg_perc,3par,ftr,orb_perc,drb_perc,trb_perc,ast_perc,stl_perc,blk_perc,tov_perc,usg_perc,ortg,drtg
0,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,0,7.0,0.580,0.538,0.165,0.405,34.2,90.0,62.8,56.4,10.9,10.4,14.7,100.0,117.6,71.9
1,"Tue, Oct 31, 2006",Chicago Bulls,Miami Heat,1,3.0,0.442,0.408,0.262,0.338,10.0,65.8,37.2,52.0,6.5,4.5,21.9,100.0,71.9,117.6
2,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,0,13.0,0.627,0.604,0.390,0.221,11.4,67.6,40.3,72.5,7.0,7.0,19.9,100.0,106.0,114.0
3,"Tue, Oct 31, 2006",Phoenix Suns,Los Angeles Lakers,1,6.0,0.609,0.590,0.145,0.289,32.4,88.6,59.7,65.2,11.0,2.1,17.6,100.0,114.0,106.0
4,"Wed, Nov 1, 2006",Indiana Pacers,Charlotte Bobcats,0,6.0,0.509,0.442,0.186,0.477,34.0,74.4,52.1,74.3,13.8,7.4,14.0,100.0,104.7,97.8


In [14]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Rename target variable
df = df.rename({'3p': 'target'}, axis=1)

# Define statistics
stats = ['ortg', 'drtg']

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40218 entries, 0 to 40217
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      40218 non-null  datetime64[ns]
 1   visitor   40218 non-null  object        
 2   home      40218 non-null  object        
 3   team      40218 non-null  object        
 4   target    40202 non-null  float64       
 5   ts_perc   40202 non-null  float64       
 6   efg_perc  40202 non-null  float64       
 7   3par      40202 non-null  float64       
 8   ftr       40202 non-null  float64       
 9   orb_perc  40202 non-null  float64       
 10  drb_perc  40202 non-null  float64       
 11  trb_perc  40202 non-null  float64       
 12  ast_perc  40202 non-null  float64       
 13  stl_perc  40202 non-null  float64       
 14  blk_perc  40202 non-null  float64       
 15  tov_perc  40202 non-null  float64       
 16  usg_perc  40202 non-null  float64       
 17  ortg      40

# Dataframe of team's last 15 performances

In [15]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [16]:
# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team'] + stats

last_15_games = df[y_cols]
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 15 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [17]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

### Last 15 Performances (Unweighted)

In [31]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15 = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15[stat] = 0
    for date in dates:
        last_15[stat] = last_15[stat] + last_15[stat + date]
    
    last_15[stat] = last_15[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15[stat + '_std'] = 0
    for date in dates:
        last_15[stat + '_std'] = last_15[stat + '_std'] + ((last_15[stat + date] - last_15[stat]) ** 2)
    
    last_15[stat + '_std'] = last_15[stat + '_std'] / len(dates)
    last_15[stat + '_std'] = last_15[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15[stat + '_trend'] = last_15[stat + '_trend'] + \
                                        z_score(last_15[stat + date], last_15[stat], last_15[stat + '_std']).fillna(0)
    
    last_15[stat + '_trend'] = last_15[stat + '_trend'] / len(dates[:10])

last_15 = last_15.groupby(['date', 'visitor', 'home']).mean()

# Standard deviation and trending cols
std_cols = [stat + '_std' for stat in stats]
trend_cols = [stat + '_trend' for stat in stats]

# Keep columns
last_15 = last_15[['target'] + stats + std_cols + trend_cols].dropna(axis=0)

# Calculate percentages
last_15['fg_perc'] = last_15['fg'] / last_15['fga']
# last_15['fg_perc_opp'] = last_15['fg_opp'] / last_15['fga_opp']

last_15['3p_perc'] = last_15['3p'] / last_15['3pa']
# last_15['3p_perc_opp'] = last_15['3p_opp'] / last_15['3pa_opp']

last_15['ft_perc'] = last_15['ft'] / last_15['fta']
# last_15['ft_perc_opp'] = last_15['ft_opp'] / last_15['fta_opp']

last_15.tail(10)

target        ortg  \
date       visitor                home                                         
2022-03-04 Minnesota Timberwolves Oklahoma City Thunder     15.5  112.840000   
           New York Knicks        Phoenix Suns              15.0  113.853333   
           Orlando Magic          Toronto Raptors           11.5  112.060000   
           Utah Jazz              New Orleans Pelicans      11.0  116.990000   
2022-03-05 Golden State Warriors  Los Angeles Lakers        16.5  113.150000   
           Orlando Magic          Memphis Grizzlies         12.5  114.590000   
           Philadelphia 76ers     Miami Heat                10.0  114.690000   
           Portland Trail Blazers Minnesota Timberwolves    15.5  113.156667   
           Sacramento Kings       Dallas Mavericks          11.5  115.876667   
           San Antonio Spurs      Charlotte Hornets         13.5  112.976667   

                                                                drtg  \
date       visitor                home                                 
2022-03-04 Minnesota Timberwolves Oklahoma City Thunder   113.636667   
           New York Knicks        Phoenix Suns            113.516667   
           Orlando Magic          Toronto Raptors         113.836667   
           Utah Jazz              New Orleans Pelicans    111.750000   
2022-03-05 Golden State Warriors  Los Angeles Lakers      115.136667   
           Orlando Magic          Memphis Grizzlies       112.626667   
           Philadelphia 76ers     Miami Heat              110.356667   
           Portland Trail Blazers Minnesota Timberwolves  115.790000   
           Sacramento Kings       Dallas Mavericks        113.810000   
           San Antonio Spurs      Charlotte Hornets       114.593333   

                                                           ortg_std  \
date       visitor                home                                
2022-03-04 Minnesota Timberwolves Oklahoma City Thunder    9.563808   
           New York Knicks        Phoenix Suns             7.680040   
           Orlando Magic          Toronto Raptors         10.717632   
           Utah Jazz              New Orleans Pelicans     9.311151   
2022-03-05 Golden State Warriors  Los Angeles Lakers       9.872557   
           Orlando Magic          Memphis Grizzlies        8.079460   
           Philadelphia 76ers     Miami Heat               9.339480   
           Portland Trail Blazers Minnesota Timberwolves  11.892380   
           Sacramento Kings       Dallas Mavericks        10.866780   
           San Antonio Spurs      Charlotte Hornets       12.347006   

                                                           drtg_std  \
date       visitor                home                                
2022-03-04 Minnesota Timberwolves Oklahoma City Thunder    9.726704   
           New York Knicks        Phoenix Suns            11.145699   
           Orlando Magic          Toronto Raptors         10.231788   
           Utah Jazz              New Orleans Pelicans    10.695041   
2022-03-05 Golden State Warriors  Los Angeles Lakers       9.811984   
           Orlando Magic          Memphis Grizzlies        8.898620   
           Philadelphia 76ers     Miami Heat              12.590394   
           Portland Trail Blazers Minnesota Timberwolves  11.661036   
           Sacramento Kings       Dallas Mavericks         9.161239   
           San Antonio Spurs      Charlotte Hornets        7.804677   

                                                          ortg_trend  \
date       visitor                home                                 
2022-03-04 Minnesota Timberwolves Oklahoma City Thunder    -0.187056   
           New York Knicks        Phoenix Suns              0.035312   
           Orlando Magic          Toronto Raptors          -0.188483   
           Utah Jazz              New Orleans Pelicans      0.400202   
2022-03-05 Golden State Warriors  Los Angeles Lakers       -0.145906   
           Orlando Mag

## Correlations

In [32]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15:
    corr_p = pearsonr(last_15['target'], last_15[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlations
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

,corr,stat
0,1.00,target
1,0.39,ortg
2,0.34,drtg
3,0.03,ortg_std
5,0.02,ortg_trend
4,0.01,drtg_std


## Save dataframe with significantly correlated stats

In [33]:
stats = corr_df[corr_df['corr'].abs() >= .3]['stat']
df = last_15[stats]
stats
# df.to_csv('backend/data/inputs/3p/advanced_totals.csv')

0    target
1      ortg
2      drtg
Name: stat, dtype: object